## Implementation of DDPG paper inspired from: [DDPG Video by Machine learning with phil](https://www.youtube.com/watch?v=jDll4JSI-xo)

## Some notes from DDPG paper for implementation purposes
- Need a replay buffer class
- Need a class for target Q network (function of s,a)
- Need to use batch norm
- Use off-policy learning (learn from behavioural policy B = mu(s)) to handle explore-exploit dilemma as policy becomes deterministic
- Deterministic policies means the algorithm outputs the actual policies instead of probabilities. Need a way to bound actions to env limits
- Two actor and two critic networks (two on-policy, two off-policy), one target network for each 
- Updates are soft, according to theta_prime = tau*theta + (1-tau)*theta_prime, with tau << 1 (0.01 or less)
- The target actor is just the evaluation actior + noise (ornstein ulhenbeck) -> need a class for noise
- Action chosen as per target network
- ai is from the action buffer
- Adam optimiser, lr = 10^-4 for actor, 10^-3 for critic
- For Q(critic), L2 weight decay of 10^-2 and gamma = 0.99
- For soft updates, tau = 0.001
- Relu for all hidden layers
- final output layer for actor -> tanh 
- action as separate input to second hidden layer of Q
- final layer weights and biases uniform distribution in a range
- theta = 0.15, sigma = 0.2 for noise (ornstein ulhenbeck)


In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.functional as F

In [2]:
# ornstein-uhlenbeck noise class
class OUNoise(object):
    def __init__(self, mu, sigma=0.15, theta=0.2,dt = 1e-2, x0=None):
        self.theta = theta
        self.mu = mu
        self.sigma = sigma
        self.dt = dt
        self.x0 = x0
        self.reset()
    
    def __call__(self):
        x = self.x_prev + self.theta * (self.mu - self.x_prev)*self.dt + \
            self.sigma * np.sqrt(self.dt)*np.random.normal(size = self.mu)
        self.x_prev = x
        return x
    
    def __reset__(self):
        self.x_prev = self.x0 if self.x0 is not None else np.zeros_like(self.mu)

In [3]:
# replay buffer class
class ReplayBuffer(object):
    def __init__(self, max_size, input_shape, n_actions):
        self.mem_size = max_size
        self.mem_counter = 0
        self.state_memory = np.zeros((self.mem_size, *input_shape))
        self.new_state_memory = np.zeros((self.mem_size, *input_shape))
        self.action_memory = np.zeros((self.mem_size, n_actions))
        self.reward_memory = np.zeros(self.mem_size)
        self.terminal_memory = np.zeros(self.mem_size, dtype=np.float32)
    
    def store_transitions(self, state, action, reward, state_, done):
        index = self.mem_counter % self.mem_size
        self.state_memory[index] = state
        self.new_state_memory[index] = state_
        self.reward_memory[index] = reward
        self.action_memory[index] = action
        self.terminal_memory[index] = 1-int(done)
        self.mem_counter+=1

    def sample_buffer(self, batch_size):
        max_memory = min(self.mem_counter, self.mem_size)
        batch = np.random.choice(max_memory, batch_size)

        states = self.state_memory[batch]
        new_states = self.new_state_memory[batch]
        actions = self.action_memory[batch]
        rewards = self.reward_memory[batch]
        terminal = self.terminal_memory[batch]

        return states, actions, rewards, new_states, terminal
    

In [4]:
# class for critic network
class CriticNetwork(nn.Module):
    def __init__(self, beta, input_dims, fc1_dims, fc2_dims, n_actions, name, chkpt_dir = 'models/ddpg'):
        super(CriticNetwork, self).__init__()

        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.chkpt_file = os.path.join(chkpt_dir, name+'_ddpg')

        # define network achitecture
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        f1 = 1/np.sqrt(self.fc1.weight.data.size()[0])
        torch.nn.init.uniform_(self.fc1.weight.data, -f1, f1)
        torch.nn.init.uniform_(self.fc1.bias.data, -f1, f1)
        self.bn1 = nn.LayerNorm(self.fc1_dims) ## apply layer normalisation

        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        f2 = 1/np.sqrt(self.fc2.weight.data.size()[0])
        torch.nn.init.uniform_(self.fc2.weight.data, -f2, f2)
        torch.nn.init.uniform_(self.fc2.bias.data, -f2,  f2)
        self.bn2 = nn.LayerNorm(self.fc2_dims) ## apply layer normalisation


        self.action_value = nn.linear(self.n_actions, self.fc2_dims)
        f3 = 0.003
        self.Q = nn.Linear(self.fc2_dims, 1)
        torch.nn.init.uniform_(self.Q.weight.data, -f3,  f3)
        torch.nn.init.uniform_(self.Q.bias.data, -f3,  f3)


        # define an optimiser
        self.optimiser = torch.optim.Adam(self.parameters, lr = beta)

        # setting model to run on a device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
    

    # define forward class
    def forward(self, state, action):
        state_value = self.fc1(state)
        state_value = self.bn1(state)
        state_value = F.relu(state_value)
        state_value = self.fc2(state_value)
        state_value = self.bn2(state_value)

        action_value = F.relu(self.action_value(action))
        state_action_value = F.relu(torch.add(state_value, action_value))
        state_action_value = self.Q(state_action_value)

        return self.Q


    # save model at checkpoint
    def save_checkpoint(self):
        print(f'saving checkpoint')
        torch.save(self.state_dict(), self.chkpt_file)

    # load model from checkpoint
    def load_checkpoint(self):
        print('loading model from checkpoint')
        self.load_state_dict(torch.load(self.chkpt_file))

In [5]:

# defining the actor network
class ActorNetwork(nn.Module):
    def __init__(self, alpha, input_dims, fc1_dims, fc2_dims, n_actions, name, chkpt_dir = 'models/ddpg'):
        super(ActorNetwork, self).__init__()
        self.input_dims = input_dims
        self.fc1_dims = fc1_dims
        self.fc2_dims = fc2_dims
        self.n_actions = n_actions
        self.chkpt_file = os.path.join(chkpt_dir, name+'_ddpg')

        
        # define network achitecture
        self.fc1 = nn.Linear(*self.input_dims, self.fc1_dims)
        f1 = 1/np.sqrt(self.fc1.weight.data.size()[0])
        torch.nn.init.uniform_(self.fc1.weight.data, -f1, f1)
        torch.nn.init.uniform_(self.fc1.bias.data, -f1, f1)
        self.bn1 = nn.LayerNorm(self.fc1_dims) ## apply layer normalisation

        self.fc2 = nn.Linear(self.fc1_dims, self.fc2_dims)
        f2 = 1/np.sqrt(self.fc2.weight.data.size()[0])
        torch.nn.init.uniform_(self.fc2.weight.data, -f2, f2)
        torch.nn.init.uniform_(self.fc2.bias.data, -f2,  f2)
        self.bn2 = nn.LayerNorm(self.fc2_dims) ## apply layer normalisation

        f3 = 0.003
        self.mu = nn.Linear(self.fc2_dims, self.n_actions)
        torch.nn.init.uniform_(self.mu.weight.data, -f3, f3)
        torch.nn.init.uniform_(self.mu.bias.data, -f3,  f3)

         # define an optimiser
        self.optimiser = torch.optim.Adam(self.parameters, lr = alpha)

        # setting model to run on a device
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.to(self.device)
    
    def forward(self, state):
        output = self.fc1(state)
        output = self.bn1(output)
        output = F.relu(output)
        output = self.fc2(output)
        output = self.bn2(output)
        output = F.relu(output)
        output = torch.tanh(self.mu(output))

        return output
    
    # save model at checkpoint
    def save_checkpoint(self):
        print(f'saving checkpoint')
        torch.save(self.state_dict(), self.chkpt_file)

    # load model from checkpoint
    def load_checkpoint(self):
        print('loading model from checkpoint')
        self.load_state_dict(torch.load(self.chkpt_file))
    

In [6]:
class Agent(object):
    def __init__(self, alpha, beta, input_dims, tau, env, gamma = 0.99, n_actions=2, max_size = 1000000,\
         layer1_size=400, layer2_size=300, batch_size=64):
         
        self.gamma = gamma
        self.tau = tau
        self.memory = ReplayBuffer(max_size, input_dims, n_actions)
        self.batch_size = batch_size

        # define actor and target actor networks
        self.actor = ActorNetwork(alpha, input_dims, layer1_size, layer2_size, n_actions, 'Actor')
        self.target_actor = ActorNetwork(alpha, input_dims, layer1_size, layer2_size, n_actions, 'TargetActor')

        # define critic and target critic networks
        self.critic = CriticNetwork(beta, input_dims, layer1_size, layer2_size, n_actions, 'Critic')
        self.target_critic = CriticNetwork(beta, input_dims, layer1_size, layer2_size, n_actions, 'TargetCritic')

        self.noise = OUNoise(mu=np.zeros(n_actions))

        self.update_network_parameters(tau=1) # solves problem of moving target
    
    def choose_action(self, observation):
        self.actor.eval()
        observation = torch.tensor(observation, dtype=torch.float).to(self.actor.device)
        




